In [9]:
# Importing required libraries

import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from datetime import datetime


from pymongo import MongoClient

In [10]:
# Creating a connection to MongoDB
client = MongoClient('192.168.30.114', 27017)
db = client['news']
collection = db['elespectador']

In [11]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [12]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
options = Options()
#options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options.headless = True
options.binary_location = '/usr/bin/firefox'
DRIVER_PATH = './geckodriver'
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("browser.privatebrowsing.autostart", True)
#DRIVER_PATH = 'I:\Escritorio\geckodriver\geckodriver.exe'

In [13]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH, options=options, firefox_profile=firefox_profile)

In [14]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [15]:
news = []
blocks = []
category = ["politica", "economia", "deportes", "bogota", "judicial"]
#category = ["politica"]

def getPath(category):
    for i in range(2):
        if i == 0:
            soup = make_request(browser, f'/archivo/{category}/')
        else:
            soup = make_request(browser, f'/archivo/{category}/{i + 1}')
        layout = soup.find(class_ = 'Layout-flexAds')
        blocks = layout.find_all('section')
        #print(len(blocks))
        cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card') 
        #print(len(cards))

        for card in cards:
            news.append({
                'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
                'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
            })
            
for i in category:
    #print(i)
    getPath(i)
    
len(news)
    #print(f"Soy el numero {i + 2}")

100

In [ ]:
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)

In [ ]:
for n in news:
    # Getting HTML content for news page
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    #date_time_str = soup.find(class_ = 'ArticleHeader-Date').get_text()
    #print(date_time_str)
    #date_time_str = rreplace(date_time_str, '.', '', p.count('.') - 1)
    #print(date_time_str)
    #date_time_obj = datetime.strptime(date_time_str, '%d/%m/%y')
    #print(date_time_obj)
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').find('a')
    if n['author'] is not None:
        #print("Existe")
        n['author'] = n['author'].get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    #n['category'] = soup.find(class_ = 'ACredit-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [ ]:
news

In [ ]:
# Storing extracted information for further analysis
collection.insert_many(news)